# Shoreline Monitor High Resolution

Hotspots + monthly temporal resolution

Notebook environment to migrate netcdf files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
# %load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [2]:
# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))
sys.path.append(r"P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\coclicodata")

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = p_drive.joinpath(
    "1000545-054-globalbeaches", "15_GlobalCoastalAtlas", "datasets"
)

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

C:\Users\heijd_de\AppData\Local\Temp\ipykernel_12936\37879644.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


<IPython.core.display.Javascript object>

In [3]:
# Project paths & files (manual input)
dataset_dir = gca_data_dir.joinpath("02_Shorelinemonitor_monthly")
dataset_dir_shorelinemonitor_hr = dataset_dir.joinpath("shorelinemonitor_monthly.nc")
dataset_out_file = "ShorelineMonitor_HR"
CF_dir = gca_data_dir.joinpath(r"CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

In [4]:
# # # write csv to netcdf

# # # Load data from CSV file into a pandas dataframe
# csv_dir = r'P:\11202268-hydraulic-engineering\MSc_students\Dante_van_der_Heijden\02. Data\shorelineMonitor_highres.csv'
# df = pd.read_csv(csv_dir)

# # Convert the pandas dataframe to an xarray dataset
# ds = xr.Dataset.from_dataframe(df)

# # Write the xarray dataset to a netCDF file
# ds.to_netcdf(dataset_dir_shorelinemonitor_hr)

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [5]:
# open datasets
dataset_shorelinemonitor_hr = xr.open_dataset(dataset_dir_shorelinemonitor_hr)

# check original dataset
dataset_shorelinemonitor_hr

<xarray.Dataset>
Dimensions:            (index: 60637)
Coordinates:
  * index              (index) int64 0 1 2 3 4 ... 60632 60633 60634 60635 60636
Data variables:
    Unnamed: 0         (index) int64 ...
    transect_id        (index) object ...
    transect_origin_x  (index) float64 ...
    transect_origin_y  (index) object ...
    transect_end_x     (index) float64 ...
    transect_end_y     (index) float64 ...
    dt                 (index) object ...
    dist               (index) object ...
    outliers           (index) object ...
    hotspot            (index) object ...
    Sdate              (index) object ...

<IPython.core.display.Javascript object>

In [6]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=dataset_dir_shorelinemonitor_hr, 
                 working_dir=CF_dir
                 )


<IPython.core.display.Javascript object>

In [7]:
# save original CF compliancy
save_compliancy(cap, testfile=dataset_dir_shorelinemonitor_hr, working_dir=CF_dir)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\02_Shorelinemonitor_monthly\\shorelinemonitor_monthly.nc': {'warnings': '22', 'errors': '7'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [8]:
import json

# NetCDF attribute alterations
f_global = open(
    r"P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\02_Shorelinemonitor_monthly\metadata_shorelinemonitor_hr.json"
)
meta_global = json.load(f_global)

for attr_name, attr_val in meta_global.items():
    if attr_name == "PROVIDERS":
        attr_val = json.dumps(attr_val)
    dataset_shorelinemonitor_hr.attrs[attr_name] = attr_val

dataset_shorelinemonitor_hr.attrs["Conventions"] = "CF-1.8"

<IPython.core.display.Javascript object>

In [9]:
dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.drop("Unnamed: 0")

<IPython.core.display.Javascript object>

In [10]:
dataset_shorelinemonitor_hr

<xarray.Dataset>
Dimensions:            (index: 60637)
Coordinates:
  * index              (index) int64 0 1 2 3 4 ... 60632 60633 60634 60635 60636
Data variables:
    transect_id        (index) object 'BOX_117_067_110' ... 'BOX_096_171_25'
    transect_origin_x  (index) float64 -71.62 -71.62 -71.63 ... -37.79 -37.79
    transect_origin_y  (index) object '10.990024' '10.991483' ... '-4.408125'
    transect_end_x     (index) float64 -71.61 -71.61 -71.62 ... -37.79 -37.79
    transect_end_y     (index) float64 11.0 11.0 11.01 ... -4.392 -4.391 -4.39
    dt                 (index) object '[0.0, 31.0, 62.0, 90.0, 396.0, 427.0, ...
    dist               (index) object '[-2.857791642818455, 6.388898734846833...
    outliers           (index) object '[24, 74, 75, 76]' ... '[109]'
    hotspot            (index) object 'prograding_feature_polygon_2167' ... '...
    Sdate              (index) object '1985-01-01 00:00:00' ... '1984-09-12 0...
Attributes: (12/21)
    TITLE:               The Shoreline Monitor High Resolution
    TITLE_ABBREVIATION:  shore_mon_hr
    DESCRIPTION:         The Shoreline Monitor High Resolution is a dataset o...
    SHORT_DESCRIPTION:   The Shoreline Monitor High Resolution is a dataset t...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 
    LONG_NAME:           THE_SHORELINE_MONITOR_HR
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [11]:
from datetime import datetime


def round_to_nearest_month(date):
    # Extract year and month from the input date
    year = date.year
    month = date.month

    # Round the day to the nearest month
    if date.day > 15:
        month += 1
        if month > 12:
            year += 1
            month = 1
    else:
        month = month

    # Create a new date object for the rounded date
    rounded_date = datetime(year, month, 1)

    return pd.to_datetime(rounded_date)

<IPython.core.display.Javascript object>

In [12]:
max_dt = pd.to_datetime('1984-01-01')

for i, x in enumerate(dataset_shorelinemonitor_hr['dt'].values):
    try:    
        lst = json.loads(x)
        if len(lst) > 0:
            max_v = int(max(lst))
            end_date = round_to_nearest_month(round_to_nearest_month(pd.to_datetime(dataset_shorelinemonitor_hr['Sdate'].values[i])) + pd.DateOffset(days = max_v))
            if end_date > max_dt:
                max_dt = end_date
    except: 
        continue


<IPython.core.display.Javascript object>

In [13]:
sdate = pd.to_datetime(min(dataset_shorelinemonitor_hr["Sdate"].values))
# edate = sdate + pd.DateOffset(max_dt)
edate = max_dt

sdate, edate

(Timestamp('1984-04-19 00:00:00'), Timestamp('2021-12-01 00:00:00'))

<IPython.core.display.Javascript object>

In [14]:
# drop time xarray

# dataset_shorelinemonitor = dataset_shorelinemonitor.drop_vars('dt')
# TODO: this needs to be changes as start_dates are different now and not always at 1984-01

date_range = pd.date_range(start=sdate, end=edate, freq="MS")
dataset_shorelinemonitor_hr["time_step"] = xr.DataArray(
    date_range, dims=["time"]
)  # TODO: altered time_range into time_step

<IPython.core.display.Javascript object>

In [15]:
start_lats = []
for x in dataset_shorelinemonitor_hr['transect_origin_y'].values:
    try:
        x2 = float(x)
    except:
        x2 = float(str(x)[:-2])
    finally:
        start_lats.append(x2)


<IPython.core.display.Javascript object>

In [16]:
# combine start and end coordinates into a transect
from shapely.geometry import LineString

start_lons = dataset_shorelinemonitor_hr["transect_origin_x"].values
end_lons = dataset_shorelinemonitor_hr["transect_end_x"].values
end_lats = dataset_shorelinemonitor_hr["transect_end_y"].values
coords = zip(zip(start_lons, start_lats), zip(end_lons, end_lats))

dataset_shorelinemonitor_hr["transect_geom"] = (
    ["index"],
    [str(LineString(line)) for line in coords],
)
dataset_shorelinemonitor_hr["transect_geom"].attrs["long_name"] = "Transect Geometry"

<IPython.core.display.Javascript object>

In [17]:
dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.assign_coords(
    transect_origin_y=start_lats
)

<IPython.core.display.Javascript object>

In [18]:
# TODO: Apply again after correct timesteps


def create_shorelinepos(time_range, positions, timesteps, sdates):
    time_range = [pd.to_datetime(date_str) for date_str in time_range]
    l = np.full((len(positions), len(time_range)), np.nan)

    for i in range(len(sdates)):
        try:
            positions_lst, timesteps_lst = np.array(json.loads(positions[i])), np.array(
                json.loads(timesteps[i])
            )
        except:
            positions_lst, timesteps_lst = [], []

        sdate = round_to_nearest_month(pd.to_datetime(sdates[i]))
        dates = [
            round_to_nearest_month(x)
            for x in pd.to_datetime(sdate) + pd.to_timedelta(timesteps_lst, unit="D")
        ]
        mask = np.isin(time_range, dates)
        indices = np.arange(len(time_range))[mask]

        l[i, indices] = positions_lst

    return l


new_var1 = xr.apply_ufunc(
    create_shorelinepos,
    dataset_shorelinemonitor_hr["time_step"].values,
    dataset_shorelinemonitor_hr["dist"].values,
    dataset_shorelinemonitor_hr["dt"].values,
    dataset_shorelinemonitor_hr["Sdate"].values,
    input_core_dims=[["time_step"], ["index"], ["index"], ["index"]],
    dask="parallelized",
    output_dtypes=[float],
    output_core_dims=[["time_step"]],
    vectorize=False,
)

<IPython.core.display.Javascript object>

In [19]:
def combine_outliers(positions, outliers):
    l = np.zeros(positions.shape)
    not_na = np.where(~np.isnan(positions))[0]
    outl = np.array(json.loads(outliers))
    if len(outl) > 0:
        l[not_na[outl]] = 1

    return l


new_var2 = xr.apply_ufunc(
    combine_outliers,
    new_var1,
    dataset_shorelinemonitor_hr["outliers"],
    input_core_dims=[["time_step"], []],
    output_core_dims=[["time_step"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[np.float32],
)

<IPython.core.display.Javascript object>

In [20]:
dataset_shorelinemonitor_hr

<xarray.Dataset>
Dimensions:            (index: 60637, transect_origin_y: 60637, time: 452)
Coordinates:
  * index              (index) int64 0 1 2 3 4 ... 60632 60633 60634 60635 60636
  * transect_origin_y  (transect_origin_y) float64 10.99 10.99 ... -4.409 -4.408
  * time               (time) datetime64[ns] 1984-05-01 ... 2021-12-01
Data variables:
    transect_id        (index) object 'BOX_117_067_110' ... 'BOX_096_171_25'
    transect_origin_x  (index) float64 -71.62 -71.62 -71.63 ... -37.79 -37.79
    transect_end_x     (index) float64 -71.61 -71.61 -71.62 ... -37.79 -37.79
    transect_end_y     (index) float64 11.0 11.0 11.01 ... -4.392 -4.391 -4.39
    dt                 (index) object '[0.0, 31.0, 62.0, 90.0, 396.0, 427.0, ...
    dist               (index) object '[-2.857791642818455, 6.388898734846833...
    outliers           (index) object '[24, 74, 75, 76]' ... '[109]'
    hotspot            (index) object 'prograding_feature_polygon_2167' ... '...
    Sdate              (index) object '1985-01-01 00:00:00' ... '1984-09-12 0...
    time_step          (time) datetime64[ns] 1984-05-01 ... 2021-12-01
    transect_geom      (index) <U61 'LINESTRING (-71.624166 10.990024, -71.60...
Attributes: (12/21)
    TITLE:               The Shoreline Monitor High Resolution
    TITLE_ABBREVIATION:  shore_mon_hr
    DESCRIPTION:         The Shoreline Monitor High Resolution is a dataset o...
    SHORT_DESCRIPTION:   The Shoreline Monitor High Resolution is a dataset t...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 
    LONG_NAME:           THE_SHORELINE_MONITOR_HR
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [21]:
delete_vars = ["dist", "outliers", "Sdate"]
for dv in delete_vars:
    dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.drop(dv)

dataset_shorelinemonitor_hr["dist"] = (["index", "time"], new_var1.data)
dataset_shorelinemonitor_hr["outliers"] = (["index", "time"], new_var2.data)

<IPython.core.display.Javascript object>

In [22]:
dataset_shorelinemonitor_hr

<xarray.Dataset>
Dimensions:            (index: 60637, transect_origin_y: 60637, time: 452)
Coordinates:
  * index              (index) int64 0 1 2 3 4 ... 60632 60633 60634 60635 60636
  * transect_origin_y  (transect_origin_y) float64 10.99 10.99 ... -4.409 -4.408
  * time               (time) datetime64[ns] 1984-05-01 ... 2021-12-01
Data variables:
    transect_id        (index) object 'BOX_117_067_110' ... 'BOX_096_171_25'
    transect_origin_x  (index) float64 -71.62 -71.62 -71.63 ... -37.79 -37.79
    transect_end_x     (index) float64 -71.61 -71.61 -71.62 ... -37.79 -37.79
    transect_end_y     (index) float64 11.0 11.0 11.01 ... -4.392 -4.391 -4.39
    dt                 (index) object '[0.0, 31.0, 62.0, 90.0, 396.0, 427.0, ...
    hotspot            (index) object 'prograding_feature_polygon_2167' ... '...
    time_step          (time) datetime64[ns] 1984-05-01 ... 2021-12-01
    transect_geom      (index) <U61 'LINESTRING (-71.624166 10.990024, -71.60...
    dist               (index, time) float64 nan nan nan ... -40.21 -40.21 nan
    outliers           (index, time) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes: (12/21)
    TITLE:               The Shoreline Monitor High Resolution
    TITLE_ABBREVIATION:  shore_mon_hr
    DESCRIPTION:         The Shoreline Monitor High Resolution is a dataset o...
    SHORT_DESCRIPTION:   The Shoreline Monitor High Resolution is a dataset t...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 
    LONG_NAME:           THE_SHORELINE_MONITOR_HR
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [23]:
dataset_shorelinemonitor_hr

<xarray.Dataset>
Dimensions:            (index: 60637, transect_origin_y: 60637, time: 452)
Coordinates:
  * index              (index) int64 0 1 2 3 4 ... 60632 60633 60634 60635 60636
  * transect_origin_y  (transect_origin_y) float64 10.99 10.99 ... -4.409 -4.408
  * time               (time) datetime64[ns] 1984-05-01 ... 2021-12-01
Data variables:
    transect_id        (index) object 'BOX_117_067_110' ... 'BOX_096_171_25'
    transect_origin_x  (index) float64 -71.62 -71.62 -71.63 ... -37.79 -37.79
    transect_end_x     (index) float64 -71.61 -71.61 -71.62 ... -37.79 -37.79
    transect_end_y     (index) float64 11.0 11.0 11.01 ... -4.392 -4.391 -4.39
    dt                 (index) object '[0.0, 31.0, 62.0, 90.0, 396.0, 427.0, ...
    hotspot            (index) object 'prograding_feature_polygon_2167' ... '...
    time_step          (time) datetime64[ns] 1984-05-01 ... 2021-12-01
    transect_geom      (index) <U61 'LINESTRING (-71.624166 10.990024, -71.60...
    dist               (index, time) float64 nan nan nan ... -40.21 -40.21 nan
    outliers           (index, time) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes: (12/21)
    TITLE:               The Shoreline Monitor High Resolution
    TITLE_ABBREVIATION:  shore_mon_hr
    DESCRIPTION:         The Shoreline Monitor High Resolution is a dataset o...
    SHORT_DESCRIPTION:   The Shoreline Monitor High Resolution is a dataset t...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 
    LONG_NAME:           THE_SHORELINE_MONITOR_HR
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [24]:
# NetCDF variable and dimension alterations

# rename or swap dimension names, the latter in case the name already exists as coordinate
dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.rename_dims(
    {"index": "stations"}
)

f_vars = open(
    r"P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\02_Shorelinemonitor_monthly\vars_shorelinemonitor_hr.json"
)
meta_vars = json.load(f_vars)

for var_name, var_dict in meta_vars.items():
    dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.rename_vars(
        {var_name: var_dict["name"]}
    )
    for key, value in var_dict.items():
        if key != "name":
            dataset_shorelinemonitor_hr[var_dict["name"]].attrs[key] = value

delete_vars = ["transect_end_x", "transect_end_y"]
for dv in delete_vars:
    dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.drop(dv)

# change dtypes
object_vars = ["transect_id", "hotspot_id", "transect_geom"]
for ov in object_vars:
    dataset_shorelinemonitor_hr[ov] = dataset_shorelinemonitor_hr[ov].astype("S")

all_vars = list(dataset_shorelinemonitor_hr.keys())
data_vars = ["sp", "outliers"]
# set some data variables to coordinates to avoid duplication of dimensions in later stage
dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.set_coords(
    [v for v in all_vars if v not in data_vars]
)

# drop index xarray
dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.drop("index")

<IPython.core.display.Javascript object>

In [25]:
dataset_shorelinemonitor_hr

<xarray.Dataset>
Dimensions:        (transect_origin_y: 60637, time: 452, stations: 60637)
Coordinates:
  * lat            (transect_origin_y) float64 10.99 10.99 ... -4.409 -4.408
  * time           (time) datetime64[ns] 1984-05-01 1984-06-01 ... 2021-12-01
    transect_id    (stations) |S15 b'BOX_117_067_110' ... b'BOX_096_171_25'
    lon            (stations) float64 -71.62 -71.62 -71.63 ... -37.79 -37.79
    times          (stations) object '[0.0, 31.0, 62.0, 90.0, 396.0, 427.0, 4...
    hotspot_id     (stations) |S31 b'prograding_feature_polygon_2167' ... b'r...
    time_step      (time) datetime64[ns] 1984-05-01 1984-06-01 ... 2021-12-01
    transect_geom  (stations) |S61 b'LINESTRING (-71.624166 10.990024, -71.60...
Dimensions without coordinates: transect_origin_y, stations
Data variables:
    sp             (stations, time) float64 nan nan nan ... -40.21 -40.21 nan
    outliers       (stations, time) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes: (12/21)
    TITLE:               The Shoreline Monitor High Resolution
    TITLE_ABBREVIATION:  shore_mon_hr
    DESCRIPTION:         The Shoreline Monitor High Resolution is a dataset o...
    SHORT_DESCRIPTION:   The Shoreline Monitor High Resolution is a dataset t...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 
    LONG_NAME:           THE_SHORELINE_MONITOR_HR
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [26]:
dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.drop(["times", "time_step"])

<IPython.core.display.Javascript object>

In [27]:
# dataset_shorelinemonitor = dataset_shorelinemonitor.rename_vars({'time_step' : 'time'})
rename = {"long_name": "Time", "standard_name": "time", "units": "yr"}

dataset_shorelinemonitor_hr["time"].attrs["long_name"] = rename["long_name"]
dataset_shorelinemonitor_hr["time"].attrs["standard_name"] = rename["standard_name"]
dataset_shorelinemonitor_hr["time"].attrs["units"] = rename["units"]

<IPython.core.display.Javascript object>

In [28]:
# save new .nc files
dataset_shorelinemonitor_hr["time"].attrs.pop("units", None)
dataset_shorelinemonitor_hr.to_netcdf(
    path=str(dataset_dir_shorelinemonitor_hr).replace(".nc", "_CF.nc")
)

<IPython.core.display.Javascript object>

In [48]:
dataset_shorelinemonitor_hr = dataset_shorelinemonitor_hr.swap_dims(
    {"transect_origin_y": "stations"}
)

<IPython.core.display.Javascript object>

In [157]:
# Define function to calculate changerate for sub-array
def calc_changerate(vary):
    varx = np.arange(0, len(vary))
    mask = ~np.isnan(vary)
    slope = -999
    if len(vary[mask]) > 0:
        slope, intercept, r_value, p_value, std_err = linregress(varx[mask], vary[mask])
    return slope * 12  # [m/month -> m/yr]


# Apply calc_changerate function to each sub-array of data in parallel
changerate = xr.apply_ufunc(
    calc_changerate,
    dataset_shorelinemonitor_hr["sp"],
    input_core_dims=[["time"]],
    output_core_dims=[[]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[np.float32],
)

<IPython.core.display.Javascript object>

In [163]:
cr_meta = {"long_name": "Changerate", "units": "m/yr"}
dataset_shorelinemonitor_hr["changerate"] = ("stations", changerate.data)

dataset_shorelinemonitor_hr["changerate"].attrs["long_name"] = cr_meta["long_name"]
dataset_shorelinemonitor_hr["changerate"].attrs["units"] = cr_meta["units"]

<IPython.core.display.Javascript object>

In [164]:
dataset_shorelinemonitor_hr

<xarray.Dataset>
Dimensions:        (stations: 60637, time: 452)
Coordinates:
    lat            (stations) float64 10.99 10.99 10.99 ... -4.41 -4.409 -4.408
  * time           (time) datetime64[ns] 1984-05-01 1984-06-01 ... 2021-12-01
    transect_id    (stations) |S15 b'BOX_117_067_110' ... b'BOX_096_171_25'
    lon            (stations) float64 -71.62 -71.62 -71.63 ... -37.79 -37.79
    hotspot_id     (stations) |S31 b'prograding_feature_polygon_2167' ... b'r...
    transect_geom  (stations) |S61 b'LINESTRING (-71.624166 10.990024, -71.60...
Dimensions without coordinates: stations
Data variables:
    sp             (stations, time) float64 nan nan nan ... -40.21 -40.21 nan
    outliers       (stations, time) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    changerate     (stations) float32 0.4072 5.371 -0.3747 ... -1.635 -0.9232
Attributes: (12/21)
    TITLE:               The Shoreline Monitor High Resolution
    TITLE_ABBREVIATION:  shore_mon_hr
    DESCRIPTION:         The Shoreline Monitor High Resolution is a dataset o...
    SHORT_DESCRIPTION:   The Shoreline Monitor High Resolution is a dataset t...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 
    LONG_NAME:           THE_SHORELINE_MONITOR_HR
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

### Check CF compliancy altered NetCDF files

In [165]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(dataset_dir_shorelinemonitor_hr).replace(".nc", "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [166]:
# save altered CF compliancy
save_compliancy(
    cap,
    testfile=str(dataset_dir_shorelinemonitor_hr).replace(".nc", "_CF.nc"),
    working_dir=CF_dir,
)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\02_Shorelinemonitor_monthly\\shorelinemonitor_monthly_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

### write data to Zarr files

In [167]:
# export to zarr in write mode (to overwrite if exists)
dataset_shorelinemonitor_hr.to_zarr(
    dataset_dir.joinpath("%s.zarr" % dataset_out_file), mode="w"
)

<IPython.core.display.Javascript object>

In [168]:
test = xr.open_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file))

<IPython.core.display.Javascript object>

In [169]:
test

<xarray.Dataset>
Dimensions:        (stations: 60637, time: 452)
Coordinates:
    hotspot_id     (stations) |S31 dask.array<chunksize=(7580,), meta=np.ndarray>
    lat            (stations) float64 dask.array<chunksize=(30319,), meta=np.ndarray>
    lon            (stations) float64 dask.array<chunksize=(30319,), meta=np.ndarray>
  * time           (time) datetime64[ns] 1984-05-01 1984-06-01 ... 2021-12-01
    transect_geom  (stations) |S61 dask.array<chunksize=(7580,), meta=np.ndarray>
    transect_id    (stations) |S15 dask.array<chunksize=(15160,), meta=np.ndarray>
Dimensions without coordinates: stations
Data variables:
    changerate     (stations) float32 dask.array<chunksize=(60637,), meta=np.ndarray>
    outliers       (stations, time) float32 dask.array<chunksize=(3790, 57), meta=np.ndarray>
    sp             (stations, time) float64 dask.array<chunksize=(3790, 57), meta=np.ndarray>
Attributes: (12/21)
    AUTHOR:              Kras, E., Van der Heijden, D. & Luijendijk, A.
    CITATION:            Kras, E. (2019). Planetary-scale classification of n...
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8
    DESCRIPTION:         The Shoreline Monitor High Resolution is a dataset o...
    ...                  ...
    SPATIAL_EXTENT:      [-180, -90, 180, 90]
    TAGS:                ['civil engineering', 'geomorphology', 'shoreline dy...
    TEMPORAL_EXTENT:     ['1984-01-01T00:00:00Z', '2022-01-01T00:00:00Z']
    TITLE:               The Shoreline Monitor High Resolution
    TITLE_ABBREVIATION:  shore_mon_hr
    UNITS:

<IPython.core.display.Javascript object>